In [9]:
import tkinter as tk
from tkinter import Message,Text
from PIL import ImageTk,Image
import cv2
import numpy as np
import time
import sys
import tensorflow as tf

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation='relu', input_shape=(100, 100, 3)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(32, (4,4), strides = 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (4,4), strides = 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (4,4), strides = 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (4,4), strides = 1, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(29, activation='softmax')
])
model.load_weights('new_weights .h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 97, 97, 16)        784       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 94, 94, 32)        8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 44, 44, 64)        32832     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 19, 19, 128)       131200    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)        

In [163]:
window = tk.Toplevel()
window.title("Sign Language Detection")
dialog_title = 'QUIT'
dialog_text = 'Are you sure?'

#window.attributes('-fullscreen', True)

window.geometry('600x600')
window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)

image1= tk.PhotoImage(file = "AMerican sign language.png")
label_for_image= tk.Label(window, image=image1)
label_for_image.pack()

#message = tk.Label(window, text="Sign Language Detector" , fg='black' ,bg="white"  ,width=18  ,height=1,font=('times', 14, 'italic bold underline'))
#message.place(x=60, y=450)

lbl3 = tk.Label(window, text="Word(s) Detected:",width=17  ,fg="black"  ,bg="white"  ,height=1 ,font=('times', 15, ' bold underline '))
lbl3.place(x=200, y=300)

message = tk.Label(window, text="" ,bg="white"  ,fg="black"  ,width=20  ,height=1, activebackground = "yellow" ,font=('times', 15, ' bold '))
message.place(x=150, y=350)

res1 = ""

In [164]:
def clear():
    global res1
    res = ""
    res1 = ""
    message.configure(text= res)
    
def pronounce():
    global res1
    from win32com.client import Dispatch 
    speak = Dispatch("SAPI.Spvoice") 
    speak.Speak(res1)
    

def real_time():
    global res1
    classes = ['a','b','c','d','del','e','f','g','h','i','j','k','l','m','n','nothing','o','p','q','r','s','space','t','u','v','w','x','y','z']

    cur_text = ''
    cap = cv2.VideoCapture(0)

    while (True):
        ret, frame = cap.read()
        cv2.rectangle(frame, (50, 50), (350, 350), (0, 255, 0), 3)

        cropped_image = frame[50:350, 50:350]
        resized_frame = cv2.resize(cropped_image, (100, 100))
        reshaped_frame = (np.array(resized_frame)).reshape((1, 100, 100, 3))
        #frame_for_model = reshaped_frame / 255
        prediction = np.array(model.predict(reshaped_frame))
        predicted_class = classes[prediction.argmax()]

        prediction_probability = prediction[0, prediction.argmax()]
        if prediction_probability > 0.2:
            predicted_class_1 = None
            if predicted_class == ' ':
                predicted_class_1 = 'space'
            else:
                predicted_class_1 = predicted_class
            cv2.putText(frame, '{} - {:.2f}%'.format(predicted_class_1, prediction_probability * 100),
                        (10, 450), 1, 2, (255, 255, 0), 2, cv2.LINE_AA)
            keypress = cv2.waitKey(1)
            if keypress == ord('s'):
                alph = predicted_class  # chr(num+96)
                cur_text += alph
                res1 += alph
                print(cur_text)
            elif keypress == ord('q'):
                break
        else:
            cv2.putText(frame, 'space', (10, 450), 1, 2, (255, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('frame', frame)

        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break

    message.configure(text = cur_text)
    cap.release()
    cv2.destroyAllWindows()

In [165]:
clearButton = tk.Button(window, text="Clear", command=clear  ,fg="red"  ,bg="white"  ,width=4  ,height=1 ,activebackground = "orange" ,font=('times', 14, ' bold '))
clearButton.place(x=410, y=345)

takeImg = tk.Button(window, text="Open Camera to detect", command = real_time, fg="black"  ,bg="yellow"  ,width=18  ,height=1, activebackground = "Red" ,font=('times', 15, ' bold '))
takeImg.place(x=190, y=400)

photo = tk.PhotoImage(file= "audio.png")
speakup = tk.Button(window, image=photo, command = pronounce, bg="white"  ,width=40  ,height=38, activebackground = "Red")
speakup.place(x=350, y=450)

quitWindow = tk.Button(window, text="Quit", command=window.destroy  ,fg="white"  ,bg="red"  ,width=10  ,height=1, activebackground = "Red" ,font=('times', 15, ' bold '))
quitWindow.place(x=200, y=450)

window.mainloop()

b
be
s
l
lo
loo
loos
loose
